In [ ]:
from scipy.io import loadmat
import numpy as np
import random
import datetime
ocr = loadmat('/Users/chang/Desktop/ocr.mat')

In [ ]:
data = np.array(ocr['data'],dtype= float)
labels = np.array(ocr['labels'],dtype= int)
testdata = np.array(ocr['testdata'],dtype= float)
testlabels = np.array(ocr['testlabels'],dtype= int)

In [ ]:
def distance(data,testdata):
    if len(data[0]) == len(testdata[0]) and data.shape[0] != 1:
        M1 = np.tile(np.transpose([(data**2).sum(axis=1)]),(1,testdata.shape[0]))
        M2 = np.tile((testdata**2).sum(axis=1),(data.shape[0],1))
        M3 = 2*np.dot(data,testdata.T)
        return M1+M2-M3
    else:
        raise Exception('invalid input')

In [ ]:
def oneNN(D,labels):
    return labels[np.array(np.argmin(D, axis=0),dtype= int)]

In [ ]:
def singlerate(preds,testlabels):
    if len(preds)==len(testlabels):
        return np.count_nonzero(preds-testlabels)/float(len(preds))
    
    else:
        raise Exception('invalid input')

In [ ]:
def pselect(data,labels):
    if (len(data)>20000) and np.mod(len(data),10) == 0:
        numrow = len(data)/10
        dataf = np.zeros((10,numrow,len(data[0])))
        for i in range(10):
            dataf[i]=data[i*numrow:(i+1)*numrow]
        D1 = np.zeros((10,10,numrow,numrow))
        D = np.zeros((len(data),len(data)))
        for i in range(10):
            for j in range(10):
                D[i*numrow:(i+1)*numrow,j*numrow:(j+1)*numrow] = distance(dataf[i],dataf[j])
    else:
        D = distance(data,data)
    sel = np.array(range(len(data)))
    seq1 = (np.tile(sel,(len(data)/200,1))).T
    seq = np.argsort(D, axis=1)[:,1:(1+len(data)/200)]
    nonze = np.resize(labels[seq],(len(data),len(data)/200))-np.resize(labels[seq1],(len(data),len(data)/200))
    return nonze

In [ ]:
nonze = pselect(data,labels)
prosel = np.array(range(len(data)))
prosel = prosel[(nonze != 0).sum(1)>0]
len(prosel)

In [ ]:
n = [1000, 2000, 4000, 8000]
def erorrate(data, labels, testdata, testlabels, n):
    errate = np.zeros((len(n), 10))
    for i in range(len(n)):
        for j in range(10):
            sel = random.sample(xrange(len(data)),n[i])
            D = distance(data[sel],testdata)
            testpred = oneNN(D, labels[sel])
            errate[i][j] = singlerate(testpred,testlabels)
    return errate

In [ ]:
errate = erorrate(data[prosel], labels[prosel], testdata, testlabels, n)
np.savetxt("temp.csv", errate, delimiter=",")